In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")


# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path, device="cuda")

In [8]:
def analyze_sentiment(sentence, aspects, absa_tokenizer, absa_model, sentiment_model):
    print(f"Sentence: {sentence}")
    print()

    # ABSA for each aspect
    for aspect in aspects:
        inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
        outputs = absa_model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        probs = probs.detach().numpy()[0]

        print(f"Sentiment of aspect '{aspect}' is:")
        for prob, label in zip(probs, ["negative", "neutral", "positive"]):
            print(f"Label {label}: {prob}")
        print()

    # Overall sentiment of the sentence
    sentiment = sentiment_model([sentence])[0]
    print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")
    print()

In [3]:
# cretae a dictionary for sentences and aspects

sentences_and_aspects = {
    "The smartphone's camera is excellent, but the battery drains quickly and the screen is fragile":
    ["camera", "battery", "screen"],

     "I love the design of this chair, but it's not very comfortable":
    ["design", "comfort"],

    "The hotel was in a great location, staff were friendly, but the rooms were dirty":
    ["location", "staff", "rooms"],

    "While the plot was intriguing, the acting was mediocre and the special effects were disappointing":
    ["plot", "acting", "special effects"],

    "The car handles well and has great fuel efficiency, but the interior feels cheap":
    ["handling", "fuel efficiency", "interior"],

    "Although the course content is comprehensive, the instructor's explanations are confusing":
    ["course content", "instructor"],

    "The restaurant offers a cozy atmosphere, but the food is overpriced and bland":
    ["atmosphere", "food"],

    "My experience with the customer service was fantastic, but the product quality is subpar":
    ["customer service", "product quality"],

    "The lecture was insightful, but the slides were poorly designed and hard to read":
    ["lecture", "slides"],

    "While the movie's soundtrack was captivating, the plot lacked depth and the ending was abrupt":
    ["soundtrack", "plot", "ending"],
}

In [9]:
for sentence, aspects in sentences_and_aspects.items():
    analyze_sentiment(sentence, aspects, absa_tokenizer, absa_model, sentiment_model)
    print('*'*100)

Sentence: The smartphone's camera is excellent, but the battery drains quickly and the screen is fragile

Sentiment of aspect 'camera' is:
Label negative: 0.0021352048497647047
Label neutral: 0.0020577849354594946
Label positive: 0.9958070516586304

Sentiment of aspect 'battery' is:
Label negative: 0.9938176274299622
Label neutral: 0.002824380761012435
Label positive: 0.0033580358140170574

Sentiment of aspect 'screen' is:
Label negative: 0.9956008195877075
Label neutral: 0.002378381323069334
Label positive: 0.0020208703354001045

Overall sentiment: negative with score 0.8199548721313477

****************************************************************************************************
Sentence: I love the design of this chair, but it's not very comfortable

Sentiment of aspect 'design' is:
Label negative: 0.05603646859526634
Label neutral: 0.042326148599386215
Label positive: 0.9016373753547668

Sentiment of aspect 'comfort' is:
Label negative: 0.9451895952224731
Label neutral: 0.03

Test for financial statements

In [10]:
financial_sentences_and_aspects = {
    "The Federal Reserve's decision to cut interest rates boosted the stock market, but inflation concerns persist":
    ["Federal Reserve", "interest rates", "stock market", "inflation"],

    "Although the technology sector performed well, energy stocks suffered due to declining oil prices":
    ["technology sector", "energy stocks", "oil prices"],

    "Despite strong earnings reports, the company's stock fell because of negative future outlook and leadership changes":
    ["earnings reports", "company's stock", "future outlook", "leadership changes"],

    "Investors are optimistic about the merger, but regulatory hurdles and antitrust issues pose significant risks":
    ["investors", "merger", "regulatory hurdles", "antitrust issues", "risks"],

    "The rising unemployment rates negatively impacted consumer spending, leading to a downturn in the retail sector":
    ["unemployment rates", "consumer spending", "retail sector"],

    "Currency fluctuations affected the company's overseas profits, but domestic sales remained strong":
    ["currency fluctuations", "overseas profits", "domestic sales"],

    "Gold prices surged as investors sought safe-haven assets amid geopolitical tensions and market uncertainty":
    ["gold prices", "safe-haven assets", "geopolitical tensions", "market uncertainty"],

    "While the IPO was successful, concerns about the company's business model and competition affected investor sentiment":
    ["IPO", "business model", "competition", "investor sentiment"],

    "New trade agreements boosted exports, but import tariffs increased production costs for manufacturers":
    ["trade agreements", "exports", "import tariffs", "production costs", "manufacturers"],

    "The housing market showed signs of recovery with increased sales, but mortgage rates are expected to rise":
    ["housing market", "sales", "mortgage rates"],
}

In [11]:
for sentence, aspects in financial_sentences_and_aspects.items():
    analyze_sentiment(sentence, aspects, absa_tokenizer, absa_model, sentiment_model)
    print('*'*100)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Sentence: The Federal Reserve's decision to cut interest rates boosted the stock market, but inflation concerns persist

Sentiment of aspect 'Federal Reserve' is:
Label negative: 0.10901546478271484
Label neutral: 0.3406997621059418
Label positive: 0.550284743309021

Sentiment of aspect 'interest rates' is:
Label negative: 0.04437537118792534
Label neutral: 0.0864158496260643
Label positive: 0.869208812713623

Sentiment of aspect 'stock market' is:
Label negative: 0.005934654735028744
Label neutral: 0.011901244521141052
Label positive: 0.9821641445159912

Sentiment of aspect 'inflation' is:
Label negative: 0.9915756583213806
Label neutral: 0.005584808997809887
Label positive: 0.002839513821527362

Overall sentiment: neutral with score 0.4659588038921356

****************************************************************************************************
Sentence: Although the technology sector performed well, energy stocks suffered due to declining oil prices

Sentiment of aspect 'tech